In [1]:
import pandas as pd


# Recalculate candles based a specific timeframe.  Data is pandas DataFrame and timeframe is in format of #min/hour/day
def resample_data(data, timeframe):
    
    resampled_data = data.resample(timeframe).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).dropna()
    
    return resampled_data
    

# Function to find high and low prices for specific time ranges within a day
def find_high_low(df, time_ranges):
    results = []
    daily_data = df.resample('D')

    for date, group in daily_data:
        intraday_results = {}
        for start_time, end_time in time_ranges:
            intraday_group = group.between_time(start_time, end_time)
            if not intraday_group.empty:
                daily_high = intraday_group['High'].max()
                daily_low = intraday_group['Low'].min()
                
                # Save results for each time range
                intraday_results[f'{start_time}-{end_time} High'] = daily_high
                intraday_results[f'{start_time}-{end_time} Low'] = daily_low

        if intraday_results:
            intraday_results['Date'] = date.date()
            intraday_results['Day of the Week'] = date.strftime('%A')
            results.append(intraday_results)

    return pd.DataFrame(results)

# Function to find the first candle closing above or below the high and low after the specified time range
def find_first_candle(df, time_ranges):
    high_low_data = find_high_low(df, time_ranges)
    
    results = []
    
    for _, row in high_low_data.iterrows():
        date = row['Date']
        day_group = df.loc[df.index.date == date]
        
        # Check for the first candle after the last time range
        for idx, candle in day_group.iterrows():
            condition = None
            if candle['Close'] > row['19:30-20:30 High']:
                condition = 'Above High'
            elif candle['Close'] < row['19:30-20:30 Low']:
                condition = 'Below Low'
            
            if condition:
                results.append({
                    'Date': idx.date(),
                    'First Candle Closing Time': idx.time(),
                    'Condition': condition,
                    'Cross Price': candle['Close'],
                    '19:30-20:30 High': row['19:30-20:30 High'],
                    '19:30-20:30 Low': row['19:30-20:30 Low'],
                    '20:30-16:00 High': row['20:30-16:00 High'],
                    '20:30-16:00 Low': row['20:30-16:00 Low']
                })
                break

    return pd.DataFrame(results)

# Function to track if price crosses the opposite bound after the first candle
def find_opposite_cross(df, time_ranges):
    first_candle_data = find_first_candle(df, time_ranges)
    results = []

    for _, row in first_candle_data.iterrows():
        date = row['Date']
        first_candle_time = row['First Candle Closing Time']
        condition = row['Condition']
        high_1930_2030 = row['19:30-20:30 High']
        low_1930_2030 = row['19:30-20:30 Low']
        high_2030_1600 = row['20:30-16:00 High']
        low_2030_1600 = row['20:30-16:00 Low']

        # Combine date and time, ensuring the correct timezone
        first_candle_timestamp = pd.Timestamp.combine(pd.to_datetime(date), first_candle_time).tz_localize('America/New_York')
        end_range = pd.Timestamp.combine(pd.to_datetime(date) + pd.Timedelta(days=1), pd.Timestamp('16:00').time()).tz_localize('America/New_York')
        
        # Filter for the time range from the first candle until 4 PM the next day
        time_filtered_group = df[(df.index >= first_candle_timestamp) & (df.index <= end_range)]
        
        # Check for the first cross to the opposite side
        if condition == 'Above High':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] <= low_1930_2030:
                    results.append({
                        'Date': idx.date(),
                        'Cross Time': idx.time(),
                        'Condition': 'Crossed Below Low',
                        'Cross Price': candle['Close'],
                        '19:30-20:30 High': high_1930_2030,
                        '19:30-20:30 Low': low_1930_2030,
                        '20:30-16:00 High': high_2030_1600,
                        '20:30-16:00 Low': low_2030_1600
                    })
                    break
        elif condition == 'Below Low':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] >= high_1930_2030:
                    results.append({
                        'Date': idx.date(),
                        'Cross Time': idx.time(),
                        'Condition': 'Crossed Above High',
                        'Cross Price': candle['Close'],
                        '19:30-20:30 High': high_1930_2030,
                        '19:30-20:30 Low': low_1930_2030,
                        '20:30-16:00 High': high_2030_1600,
                        '20:30-16:00 Low': low_2030_1600
                    })
                    break

    return pd.DataFrame(results)

# Combine the results into one DataFrame
def combine_results_with_cross(df, time_ranges):
    high_low_data = find_high_low(df, time_ranges)
    cross_data = find_opposite_cross(df, time_ranges)
    
    # Merge the high/low data with the cross data on the Date column
    combined_df = pd.merge(high_low_data, cross_data, on='Date', how='left')

    return combined_df



In [2]:
df = pd.read_csv(f'Futures Asset Data/{'MNQ=F'.split("=")[0]}.csv', index_col=False)
df = df.drop(columns=['Adj Close', 'Volume'])
df = pd.DataFrame(df)
df_copy = df


df_copy['Datetime'] = pd.to_datetime(df['Datetime'])
# Ensure the Datetime column is converted to timezone-naive
# df_copy['Datetime'] = df_copy['Datetime'].dt.tz_localize(None)
df_copy.set_index('Datetime', inplace=True)

In [3]:
# Define the two time ranges
time_ranges = [('19:30', '20:30'), ('20:30', '16:00')]

# Get the combined results with cross data
combined_df = combine_results_with_cross(df_copy, time_ranges)
combined_df = combined_df.drop(columns=['19:30-20:30 High_x', '19:30-20:30 Low_x', '20:30-16:00 High_x' , '20:30-16:00 Low_x'])

In [4]:
combined_df.head(5)

,Date,Day of the Week,Cross Time,Condition,Cross Price,19:30-20:30 High_y,19:30-20:30 Low_y,20:30-16:00 High_y,20:30-16:00 Low_y
0,2024-03-10,Sunday,21:41:00,Crossed Above High,18049.00,18048.00,18012.00,18055.50,17990.50
1,2024-03-11,Monday,20:34:00,Crossed Above High,18060.25,18060.00,18035.75,18065.50,17890.00
2,2024-03-12,Tuesday,19:23:00,Crossed Above High,18487.50,18486.75,18471.75,18491.00,17930.00
3,2024-03-13,Wednesday,09:47:00,Crossed Below Low,18353.25,18377.50,18353.50,18504.25,18281.50
4,2024-03-14,Thursday,10:00:00,Crossed Below Low,18210.75,18252.75,18223.75,18428.25,18161.25


In [5]:
TickerList = (
    "MES=F", "MYM=F", "NG=F", "MNQ=F", 
    "ES=F", "NQ=F", "YM=F", "CL=F", 
    "MCL=F", "GC=F", "MGC=F", "RTY=F", 
    "M2K=F", "SI=F", "PL=F", "HG=F", "SIL=F"
    ) 

In [6]:
for ticker in TickerList:
    df = pd.read_csv(f'Futures Asset Data/{ticker.split("=")[0]}.csv', index_col=False)
    df = df.drop(columns=['Adj Close', 'Volume'])
    df = pd.DataFrame(df)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    
    # Define the two time ranges
    time_ranges = [('19:30', '20:30'), ('20:30', '16:00')]

    # Call the function
    combined_df = combine_results_with_cross(df_copy, time_ranges)

    combined_df = combined_df.drop(columns=['19:30-20:30 High_x', '19:30-20:30 Low_x', '20:30-16:00 High_x' , '20:30-16:00 Low_x'])

    # Write the .csv files
    combined_df.to_csv(f'Backtest Results/1 Minute Timeframe Results/{ticker.split('=')[0]}.csv', index=False)

    print(ticker)

MES=F
MYM=F
NG=F
MNQ=F
ES=F
NQ=F
YM=F
CL=F
MCL=F
GC=F
MGC=F
RTY=F
M2K=F
SI=F
PL=F
HG=F
SIL=F
